**Projeto de Disciplina:** Analise de dados com Spark  
**Aluno:** Raphael da Rocha Fonseca

**Fonte dos dados:** https://www.kaggle.com/datasets/fazilbtopal/auto85/code  
**Sobre a base:** Contém atributos de carros do ano de 1985  
**Objetivo:** Montar um algoritmo de regressão linear para prever o preço do carro baseado em atributos da base  
**Infraestrutura:** Criado notebook na plataforma Databricks Comunity e utilizado Storage da própria plataforma

#### Importando as bibliotecas

In [0]:
from pyspark.sql.functions import col, lit
import pyspark.pandas as pd

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.regression import LinearRegression, LinearRegressionTrainingSummary
from pyspark.ml.evaluation import RegressionEvaluator

#### Importando o arquivo e definindo nome das colunas

In [0]:
df_carros = spark.read.csv('/FileStore/tables/auto.csv', inferSchema=True)
df_carros = df_carros.toDF(*['symboling','normalized_losses','make','fuel_type',
                            'aspiration','num_of_doors','body-style','drive_wheels',
                            'engine_location','wheel_base','length','width','height',
                            'curb_weight','engine_type','num_of_cylinders','engine_size',
                            'fuel_system','bore','stroke','compression_ratio','horsepower',
                            'peak_rpm','city_mpg','highway_mpg','price'])

df_carros.printSchema()

root
 |-- symboling: integer (nullable = true)
 |-- normalized_losses: string (nullable = true)
 |-- make: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- num_of_doors: string (nullable = true)
 |-- body-style: string (nullable = true)
 |-- drive_wheels: string (nullable = true)
 |-- engine_location: string (nullable = true)
 |-- wheel_base: double (nullable = true)
 |-- length: double (nullable = true)
 |-- width: double (nullable = true)
 |-- height: double (nullable = true)
 |-- curb_weight: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- num_of_cylinders: string (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- fuel_system: string (nullable = true)
 |-- bore: string (nullable = true)
 |-- stroke: string (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- peak_rpm: string (nullable = true)
 |-- city_mpg: integer (nu

In [0]:
display(df_carros)

symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body-style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
2,?,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
1,158,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
1,?,audi,gas,std,four,wagon,fwd,front,105.8,192.7,71.4,55.7,2954,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920
1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
0,?,audi,gas,turbo,two,hatchback,4wd,front,99.5,178.2,67.9,52.0,3053,ohc,five,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?


#### Algumas colunas definem o missing value como "?" fazendo replace para Null e depois dropando as linhas

In [0]:
df_carros = df_carros.na.replace('?', None).dropna()

#### Transformando dados categoricos da coluna num_of_cylinders em contínuos

In [0]:
dict_cilindros = {
  'two' : '2',
  'three' : '3',
  'four' : '4',
  'five' : '5',
  'six' : '6',
  'seven' : '7',
  'eight' : '8',
  'nine' : '9',
  'ten' : '10',
  'eleven' : '11',
  'twelve' : '12'
}


df_carros = df_carros.na.replace(dict_cilindros, subset=['num_of_cylinders'])

#### Transformando em Inteiro colunas numericas que estavam como string

In [0]:
df_carros = df_carros.withColumn('num_of_cylinders', col('num_of_cylinders').cast('int'))\
                     .withColumn('price', col('price').cast('int'))\
                     .withColumn('horsepower', col('horsepower').cast('int'))\
                     .withColumn('normalized_losses', col('normalized_losses').cast('int'))\
                     .withColumn('bore', col('bore').cast('int'))\
                     .withColumn('stroke', col('stroke').cast('int'))\
                     .withColumn('peak_rpm', col('peak_rpm').cast('int'))

#### Selecionando apenas as colunas contínuas para utilização na Regressão

In [0]:
colunas_continuas = [name for name, dtype in df_carros.dtypes if dtype != 'string']
df_carros_select = df_carros.select(colunas_continuas).drop('symboling')
df_carros_select.printSchema()

root
 |-- normalized_losses: integer (nullable = true)
 |-- wheel_base: double (nullable = true)
 |-- length: double (nullable = true)
 |-- width: double (nullable = true)
 |-- height: double (nullable = true)
 |-- curb_weight: integer (nullable = true)
 |-- num_of_cylinders: integer (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- bore: integer (nullable = true)
 |-- stroke: integer (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- peak_rpm: integer (nullable = true)
 |-- city_mpg: integer (nullable = true)
 |-- highway_mpg: integer (nullable = true)
 |-- price: integer (nullable = true)



#### Verificando a correlação de cada coluna com a coluna de Preço

In [0]:
df_corr = df_carros_select.to_pandas_on_spark().corr().reset_index()
df_corr[['index', 'price']]

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field pearson(__correlation_output__). If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: MatrixUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,index,price
0,normalized_losses,0.202761
1,wheel_base,0.734419
2,length,0.760952
3,width,0.843371
4,height,0.244836
5,curb_weight,0.893639
6,num_of_cylinders,0.641578
7,engine_size,0.841496
8,bore,0.313068
9,stroke,0.091693


#### Filtrando apenas as colunas com correlação maior que 70%

In [0]:
df_corr[ abs(df_corr["price"]) > 0.7 ][['index','price']]

,index,price
1,wheel_base,0.734419
2,length,0.760952
3,width,0.843371
5,curb_weight,0.893639
7,engine_size,0.841496
11,horsepower,0.759874
14,highway_mpg,-0.720090
15,price,1.000000


#### Selecionando as colunas que serão usadas na criação do modelo

In [0]:
df_regressao = df_carros_select.select('price', 'curb_weight', 'engine_size', 'horsepower', 'highway_mpg')

#### Criando uma Coluna Feature que contém uma série com os valores das colunas a serem usadas na regressão

In [0]:
assembler = VectorAssembler(inputCols=["curb_weight", "engine_size", "horsepower", "highway_mpg"],outputCol="features")
df_regressao_feature = assembler.transform(df_regressao)\
                                .selectExpr('price as label', 'features')

df_regressao_feature.display()

label,features
13950,"Map(vectorType -> dense, length -> 4, values -> List(2337.0, 109.0, 102.0, 30.0))"
17450,"Map(vectorType -> dense, length -> 4, values -> List(2824.0, 136.0, 115.0, 22.0))"
17710,"Map(vectorType -> dense, length -> 4, values -> List(2844.0, 136.0, 110.0, 25.0))"
23875,"Map(vectorType -> dense, length -> 4, values -> List(3086.0, 131.0, 140.0, 20.0))"
16430,"Map(vectorType -> dense, length -> 4, values -> List(2395.0, 108.0, 101.0, 29.0))"
16925,"Map(vectorType -> dense, length -> 4, values -> List(2395.0, 108.0, 101.0, 29.0))"
20970,"Map(vectorType -> dense, length -> 4, values -> List(2710.0, 164.0, 121.0, 28.0))"
21105,"Map(vectorType -> dense, length -> 4, values -> List(2765.0, 164.0, 121.0, 28.0))"
5151,"Map(vectorType -> dense, length -> 4, values -> List(1488.0, 61.0, 48.0, 53.0))"
6295,"Map(vectorType -> dense, length -> 4, values -> List(1874.0, 90.0, 70.0, 43.0))"


#### Separando o conjunto em Training e Teste, aplicando seed para reprodutibilidade

In [0]:
(trainingData, testData) = df_regressao_feature.randomSplit([0.7, 0.3], seed=1234567)
print(trainingData.count(), testData.count())

118 41


#### Criação do Modelo de Regressão Linear com base no conjunto de Treino

In [0]:
lrModelo = LinearRegression().fit(trainingData)

In [0]:
resultado = lrModelo.summary
print(f"Coefficients: {lrModelo.coefficients}")
print(f"Intercept: {lrModelo.intercept}")
print(resultado.meanSquaredError)
print(resultado.rootMeanSquaredError)
print(resultado.r2)

Coefficients: [7.138934531408429,52.07088235717432,15.716416449568996,20.439812776444132]
Intercept: -14501.367766573252
7366364.939203544
2714.10481359942
0.8014212917254471


#### Aplicando modelo no conjunto de Teste

In [0]:
df_resultado = lrModelo.transform(testData)
df_resultado.display()

label,features,prediction
5389,"Map(vectorType -> dense, length -> 4, values -> List(1918.0, 92.0, 68.0, 41.0))",5888.378483933053
5399,"Map(vectorType -> dense, length -> 4, values -> List(1837.0, 79.0, 60.0, 42.0))",4527.911797425599
6095,"Map(vectorType -> dense, length -> 4, values -> List(1900.0, 91.0, 68.0, 38.0))",5646.487341681195
6649,"Map(vectorType -> dense, length -> 4, values -> List(1918.0, 97.0, 69.0, 37.0))",6082.690061062718
6695,"Map(vectorType -> dense, length -> 4, values -> List(1945.0, 91.0, 68.0, 38.0))",5967.739395594575
6795,"Map(vectorType -> dense, length -> 4, values -> List(1905.0, 91.0, 68.0, 38.0))",5682.182014338237
6849,"Map(vectorType -> dense, length -> 4, values -> List(1938.0, 97.0, 69.0, 37.0))",6225.468751690887
7126,"Map(vectorType -> dense, length -> 4, values -> List(2145.0, 108.0, 82.0, 37.0))",8480.321319465746
7198,"Map(vectorType -> dense, length -> 4, values -> List(2109.0, 98.0, 70.0, 37.0))",7514.013855368468
7349,"Map(vectorType -> dense, length -> 4, values -> List(2024.0, 97.0, 69.0, 37.0))",6839.417121392014


#### Avaliando métricas de Erro

In [0]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(df_resultado)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")

Root Mean Squared Error (RMSE) on test data: 2027.8904641674626


#### RMSE Training vs Teste

In [0]:
print('Treinamento:', round(resultado.rootMeanSquaredError,2), '\nTeste:', round(rmse,2), 
      '\nDiff:', round(resultado.rootMeanSquaredError - rmse,2) )

Treinamento: 2714.1 
Teste: 2027.89 
Diff: 686.21


#### Proposta para melhorar desempenho do modelo

O modelo foi treinado com uma base de apenas 118 linhas, seria interessante treiná-lo em uma base de dados mais volumosa